In [36]:
import numpy as np
import scipy.stats as st
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import YearBegin
from pandas.tseries.offsets import MonthBegin

In [69]:
def metropolis_hastrings(log_posteior_function, initial, a_vector_idx, c_vector_idx, 
                         p_vector_idx, s_vector_idx, a_sample_idx, c_sample_idx, 
                         p_sample_idx, a_max, c_max, p_max, N, D, 
                         scale=[0.013, 0.013, 0.013, 0.013, 1, 1], 
                         shape=100, rate=2, iteration=10, seed=99999):
    
    current = initial    
    samples = np.zeros((iteration, initial.shape[0]))
    scale_vector = np.zeros(current.shape[0])
    max_ = 0
    acceptance_ = 0
    
    #set scale_vector for a, c, p, s, kappa and mu
    for i, max_index in enumerate((a_max, c_max, p_max)):       
        scale_vector[max_:(max_+max_index)] = scale[i]*np.ones(max_index)
        max_ += max_index
    scale_vector[-16:-4] = scale[3]*np.ones(12)
    scale_vector[-4:-1] = scale[4]*np.ones(3)
    scale_vector[-1] = scale[5]
    
    np.random.seed(seed=seed)
    for i in range(iteration):        
        candidate = np.zeros(current.shape[0])      
        #random draw a, c, p, s
        candidate = current + scale_vector*np.random.normal(size=current.shape[0])       
        
        logPosterior_current = log_posteior_function(params=current, a_vector_idx=a_vector_idx, 
                                      c_vector_idx=c_vector_idx, p_vector_idx=p_vector_idx, 
                                      s_vector_idx=s_vector_idx, a_sample_idx=a_sample_idx, 
                                      c_sample_idx=c_sample_idx, p_sample_idx=p_sample_idx, 
                                      N=N, D=D)
        logPosterior_candidate = log_posteior_function(params=candidate, a_vector_idx=a_vector_idx, 
                                      c_vector_idx=c_vector_idx, p_vector_idx=p_vector_idx, 
                                      s_vector_idx=s_vector_idx, a_sample_idx=a_sample_idx, 
                                      c_sample_idx=c_sample_idx, p_sample_idx=p_sample_idx, 
                                      N=N, D=D)
        
        if logPosterior_candidate - logPosterior_current > np.log(np.random.rand()):
            current = candidate
            acceptance_ += 1
        
        samples[i, :] = current
        acceptance_rate = acceptance_/iteration
    
    return (samples, acceptance_rate)  

In [4]:
def logPosterior(params, a_vector_idx, c_vector_idx, p_vector_idx, s_vector_idx,
                 a_sample_idx, c_sample_idx, p_sample_idx, N, D, shape=100, rate=2): 
       
    if np.absolute(params[-1]) > 100:
        eta = np.NINF
    else:
        eta = params[a_vector_idx] + params[c_vector_idx] + params[p_vector_idx] + \
                params[s_vector_idx] + params[-1]

    log_posterior = logKappa(params[-4:-1], shape, rate) + logPrior(params[a_sample_idx],
                    params[c_sample_idx], params[p_sample_idx],
                    params[-4:-1]) + logLikelihood(N, D, eta)
    
    return log_posterior

In [5]:
def logKappa(kappa, shape=100, rate=2):
    if np.any(kappa <= 0):
        return np.NINF
    else:
        return np.sum((shape-1)*np.log(kappa) - rate*kappa)

In [6]:
def logPrior(a_sample, c_sample, p_sample, kappa):
    
    log_prior = 0       
    for i, sample in enumerate((a_sample, c_sample, p_sample)):
        if np.all(sample < 50):
            log_prior += np.log(kappa[i])*(sample.shape[0]-1)/2 - \
                         kappa[i]*np.sum(np.power((sample[1:]-sample[:-1]), 2))/2
        else:
            log_prior += np.NINF
            
    return log_prior

In [7]:
def logLikelihood(N, D, eta):

    #calculate log_eta with method to prevent overflow    
    log_p = np.zeros(eta.shape[0])
    log_p[eta>=0] = -1*np.log(1+np.exp(-1*eta[eta>=0]))
    log_p[eta<0] = eta[eta<0] - np.log(1+np.exp(eta[eta<0]))
    
    log_q = np.zeros(eta.shape[0])
    log_q[eta>=0] = -1*eta[eta>=0] - np.log(1+np.exp(-1*eta[eta>=0]))
    log_q[eta<0] = -1*np.log(1+np.exp(eta[eta<0]))
    
    log_likelihood = np.sum(N*log_p + (D-N)*log_q)
    
    return log_likelihood

In [9]:
def initialization(data, age_col, cohort_col, period_col, seasonality=None, shape=100, rate=2,):
    
    data_ = data.dropna()
    component_idx = {} #dictionary for the index of a, c, p and s
    
    component_idx['age'] = data_[age_col].astype(int).values
    component_idx['cohort'] = (data_[cohort_col] - data_[cohort_col].min()).astype(int).values
    component_idx['period'] = (data_[period_col] - data_[period_col].min()).astype(int).values
    
    if seasonality is not None:
        component_idx['seasonality'] = (data_[period_col].dt.strftime('%m').astype(int)).values
    else:
        component_idx['seasonality'] = np.zeros(data_.shape[0])
    
    sample_vector_length = component_idx['age'].max() + component_idx['cohort'].max() + \
                            component_idx['period'].max() + 16 # 12 for seasonality, 3 or kappa and 1 for mu
    
    #sample vector contains a, c, p, s, kappa and mu. a, c, p contains values that may not have a correspoding
    #one in the index from the data, as 1:max is all created
    init = np.zeros(sample_vector_length)  #initialize everything to 0
    init[-4:-1] = np.random.gamma(shape, 1/rate, 3) #initialize kappa with random draw from gamma distribution
    
    return (init, component_idx)

In [64]:
data = pd.read_csv('C:\\Users\\LI09432\\Documents\\BayesianAPC\\data\\Holford1983Biometrics.csv', parse_dates=[1], 
                   skiprows=5)
data['time'] = pd.to_datetime(data['time'], format="%Y-%m")
data['period'] = data['time'].dt.to_period('M')
data['cohort'] = data['time'] - pd.to_timedelta(data['age'].astype(int), unit='Y')
data['cohort'] = data['cohort'] + MonthBegin()
data['cohort'] = data['cohort'].dt.to_period('M')
data.head()

,time,age,mortality,population,period,cohort
0,1935-01-01,50,177,301000,1935-01,1885-01
1,1935-01-01,55,262,212000,1935-01,1880-01
2,1935-01-01,60,360,159000,1935-01,1875-01
3,1935-01-01,65,409,132000,1935-01,1870-01
4,1935-01-01,70,328,76000,1935-01,1865-01


In [66]:
%%time

#initialize sample vector and index vector
init, idx = initialization(data, 'age', 'cohort', 'period')

#sample_idx contains the idx of sorted unique numerical a, c, p
#vector_idx contains the idx of numerical a, c, p and s in the train sample
#idx_max contains the max of a, c, and p
#notice that sample_idx is from 0 to max for each component but vector_idx doesn't
#necessarily contain all the sample_idx (e.g. some cohorts are missing)
sample_idx = {}
vector_idx = {}
idx_max = np.zeros(3)

for i, component in enumerate(['age', 'cohort', 'period', 'seasonality']):   
     if i < 3:
        sample_idx[component] = (np.unique(idx[component]) + np.sum(idx_max)).astype(int)
        vector_idx[component] = (idx[component] + np.sum(idx_max)).astype(int)
        idx_max[i] = idx[component].max()
     else:
        vector_idx[component] = (idx[component] + np.sum(idx_max) - 1).astype(int)

idx_max = idx_max.astype(int)
N = data['mortality'].values
D = data['population'].values

Wall time: 19.1 ms


In [70]:
%%time
result, acceptance_rate = metropolis_hastrings(log_posteior_function=logPosterior, initial=init, a_vector_idx=vector_idx['age'],
                      c_vector_idx=vector_idx['cohort'], p_vector_idx=vector_idx['period'], 
                    s_vector_idx=vector_idx['seasonality'], a_sample_idx=sample_idx['age'], 
                    c_sample_idx=sample_idx['cohort'], p_sample_idx=sample_idx['period'], a_max=idx_max[0],
                      c_max=idx_max[1], p_max=idx_max[2], scale=[0.005, 0.005, 0.005, 0.005, 0.01, 0.01], 
                                               N=N, D=D, iteration=50000)

Wall time: 20.3 s
